<h1>Image Colorization</h1>
<p>This is our experimantation on using TensorFlow to predict colored images from black and white</p>

In [1]:
# import libraries
import os

import numpy as np
import tensorflow as tf
from skimage.color import lab2rgb, rgb2lab
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, InputLayer, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import (array_to_img, img_to_array,load_img)
from tqdm import tqdm

In [2]:
# get L value (this is our X)
def get_lab(img):
    l = rgb2lab(img/255)[:,:,0] #Just the first layer which is the L layer
    return l

In [8]:
# get A & B values (this is our Y)
def get_color(img):
    x = rgb2lab(img/255)[:,:,1:] # this is the A and B values; a-magenta-green; b-yellow-blue
    x/=128
    return x

<p>
    We need to rewrite the how the images are saved to be np[lenght, width, layers(1,2), channels(total number of images)
</p>

In [22]:
# this uploads our images
def get_images(path, color="lab"):
    images = list()
    for filename in os.listdir(path):
        if filename[0] != '.':
            if color == "lab":
                img = get_lab(np.array(img_to_array(load_img(path + filename)), dtype=float))
                images.append(img.reshape(1,img.shape[0],img.shape[1],1))
            else:
                img = get_color(np.array(img_to_array(load_img(path + filename)), dtype=float))
                images.append(img.reshape(1,img.shape[0],img.shape[1],2))
    return images

In [25]:
# our array needs to look like (height, width, layers, number of images)
#l value only (data)
x = get_images("./OurTrainingImages/")
#a and b values (targets)
y = get_images("./OurTrainingImages/", color="yes")

In [28]:
print(y)

[array([[[[-2.81192338e-02, -2.04057251e-01],
         [-2.81192338e-02, -2.04057251e-01],
         [-2.81192338e-02, -2.04057251e-01],
         ...,
         [-2.45097376e-02, -1.30406151e-01],
         [-3.34354561e-02, -1.28828643e-01],
         [-3.34609597e-02, -1.28714878e-01]],

        [[-2.82697507e-02, -2.03880055e-01],
         [-2.82697507e-02, -2.03880055e-01],
         [-2.82697507e-02, -2.03880055e-01],
         ...,
         [-3.34354561e-02, -1.28828643e-01],
         [-3.34354561e-02, -1.28828643e-01],
         [-3.34609597e-02, -1.28714878e-01]],

        [[-2.84179198e-02, -2.03703633e-01],
         [-2.84179198e-02, -2.03703633e-01],
         [-2.84179198e-02, -2.03703633e-01],
         ...,
         [-3.34354561e-02, -1.28828643e-01],
         [-3.34354561e-02, -1.28828643e-01],
         [-3.34609597e-02, -1.28714878e-01]],

        ...,

        [[ 6.69551574e-03, -1.87754621e-03],
         [-1.21586204e-02,  3.84752198e-04],
         [-3.37002562e-02,  7.2210808

In [10]:
# create model
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1, channels))) # input shape is only needed for first layer? input_shape=(256, 256, 3)
# 3x3 kernel used and 8 filters?
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
# figure out what this does
# model.add(layers.MaxPooling2D((2, 2)))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3,3), activation='tanh', padding='same'))
# get working after we get NN working better
'''
# supposed to soften image
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
'''
# get summary of layers and compile
model.summary()
model.compile(optimizer='adam',loss='mse') # loss='sparse_categorical_crossentropy', optomizer='rmsprop'


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, None, None, 8)     80        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 16)    1168      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 32)    9248      
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, None, None, 32)    0         
_________________________________________

In [ ]:
# there is an issue fitting the data
for e in tqdm(range(10000)):
    for i,j in enumerate(x):
        model.fit(x=x[i],y=y[i], batch_size=50,verbose=0, epochs=1)


In [ ]:
# save model
model.save('./img_predictions/model.h5') 

In [6]:
#Load test images
test_images = get_images("./OurTrainingImages/")
# print(len(test_images))

for i,z in enumerate(test_images):
    # make predictions
    output = model.predict(z)
    output*=128
    cur = np.zeros((256,256,3))
    cur[:,:,0] = z[:,:,0] # L layer?
    cur[:,:,1:] = output[0] # A B layers?
    rgb_image = lab2rgb(cur)

    img = array_to_img(rgb_image)
    img.save("./img_predictions/{}.jpg".format(i))
    img.show() 


522
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, None, None, 8)     80        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 16)    1168      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 32)    9248      
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, None, None, 32)    0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 32)    9

  0%|                                                                            | 5/10000 [03:27<115:30:56, 41.61s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\einel\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-3eae066b4e29>", line 45, in <module>
    model.fit(x=x[i],y=y[i], batch_size=50,verbose=0, epochs=1)
  File "C:\Users\einel\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 727, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\einel\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_arrays.py", line 675, in fit
    steps_name='steps_per_epoch')
  File "C:\Users\einel\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_arrays.py", line 394, in model_iteration
    batch_outs = f(ins_batch)
  File "C:\Users\einel\Anaconda3\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3476, in __call__
    run_metadata=self.run_metadata)
  File "C:\Users\einel\Anaconda3\li

KeyboardInterrupt: 